In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

In [2]:
# '네이버 영화 리뷰 데이터' 로 검색
train_df = pd.read_csv("https://raw.githubusercontent.com/e9t/nsmc/master/ratings_train.txt", sep='\t')
test_df = pd.read_csv("https://raw.githubusercontent.com/e9t/nsmc/master/ratings_test.txt", sep='\t')

train_df.head(3)

,id,document,label
0,9976970,아 더빙.. 진짜 짜증나네요 목소리,0
1,3819312,흠...포스터보고 초딩영화줄....오버연기조차 가볍지 않구나,1
2,10265843,너무재밓었다그래서보는것을추천한다,0


In [3]:
train_df.shape, test_df.shape

((150000, 3), (50000, 3))

### 1. 데이터 전처리

- train data set

In [4]:
# 중복 여부 확인
train_df.document.nunique()

146182

In [5]:
# 중복 데이터 제거
train_df.drop_duplicates(subset=['document'], inplace=True)
train_df.shape

(146183, 3)

In [6]:
# 결측치 있는지 확인
train_df.isnull().sum()

id          0
document    1
label       0
dtype: int64

In [7]:
# 결측치 제거
train_df = train_df.dropna(how='any')
train_df.shape

(146182, 3)

In [8]:
train_df.label.value_counts()  # 확인

0    73342
1    72840
Name: label, dtype: int64

- test data set

In [9]:
# 중복 여부 확인
test_df.document.nunique()

49157

In [10]:
# 중복 데이터 제거
test_df.drop_duplicates(subset=['document'], inplace=True)
test_df.shape

(49158, 3)

In [11]:
# 결측치 있는지 확인
test_df.isnull().sum()

id          0
document    1
label       0
dtype: int64

In [12]:
# 결측치 제거
test_df = test_df.dropna(how='any')
test_df.shape   # 확인

(49157, 3)

### 2. 텍스트 전처리

- train data set

In [13]:
import warnings
warnings.filterwarnings(action='ignore')

In [14]:
# 한글과 공백 이외는 제거
train_df['document'] = train_df.document.str.replace('[^ㄱ-ㅎㅏ-ㅣ가-힣 ]','')
train_df.head(3)

,id,document,label
0,9976970,아 더빙 진짜 짜증나네요 목소리,0
1,3819312,흠포스터보고 초딩영화줄오버연기조차 가볍지 않구나,1
2,10265843,너무재밓었다그래서보는것을추천한다,0


In [15]:
# '' 데이터는 Nan으로 변환한 후 제거
train_df['document'].replace('', np.nan, inplace=True)
train_df.document.isnull().sum()

391

In [16]:
# 결측치 제거
train_df = train_df.dropna(how='any')
train_df.shape

(145791, 3)

- test data set

In [17]:
test_df['document'] = test_df.document.str.replace('[^ㄱ-ㅎㅏ-ㅣ가-힣 ]','')

In [18]:
test_df['document'].replace('', np.nan, inplace=True)
test_df.document.isnull().sum()

162

In [19]:
test_df = test_df.dropna(how='any')
test_df.shape

(48995, 3)

- 새로운 파일로 저장

In [51]:
train_df.to_csv('naver_movie_train.tsv', sep = '\t', index = False)
test_df.to_csv('naver_movie_test_df.tsv', sep = '\t', index = False)

### 3. 한글 처리

In [21]:
from konlpy.tag import Okt
okt = Okt()

In [22]:
stopwords = ['은','는','이','가','의','들','좀','잘','걍','과','도','을','를','으로','자','에','와','한','하다']

In [23]:
text = '교도소 이야기구먼 솔직히 지루하고 재미는 없음평점 조정해야할듯'
okt.morphs(text)

['교도소', '이야기', '구먼', '솔직히', '지루하고', '재미', '는', '없음', '평점', '조정', '해야', '할듯']

In [24]:
okt.morphs(text, stem=True)

['교도소', '이야기', '구먼', '솔직하다', '지루하다', '재미', '는', '없다', '평점', '조정', '하다', '하다']

In [37]:
from tqdm import tqdm_notebook
tqdm_notebook

  0%|          | 0/34 [00:00<?, ?it/s]

In [25]:
# X_train
from tqdm import tqdm_notebook
X_train = []

for sentence in tqdm_notebook(train_df.document):
    morphs = okt.morphs(sentence, stem=True)
    temp_X = ' '.join([word for word in morphs if not word in stopwords])
    X_train.append(temp_X)

  0%|          | 0/145791 [00:00<?, ?it/s]

In [26]:
# X_test
from tqdm import tqdm_notebook
X_test = []

for sentence in tqdm_notebook(test_df.document):
    morphs = okt.morphs(sentence, stem=True)
    temp_X = ' '.join([word for word in morphs if not word in stopwords])
    X_test.append(temp_X)

  0%|          | 0/48995 [00:00<?, ?it/s]

In [27]:
y_train = train_df.label.values
y_test = test_df.label.values

### 4. Feature 변환, 모델 학습/예측/평가

In [28]:
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score

In [29]:
cvect = CountVectorizer()
cvect.fit(X_train)
X_train_cv = cvect.transform(X_train)
X_test_cv = cvect.transform(X_test)

In [30]:
lr = LogisticRegression()
lr.fit(X_train_cv, y_train)

LogisticRegression()

In [31]:
pred = lr.predict(X_test_cv)
accuracy_score(y_test, pred)

0.8261659352995203

### 5. 실제 테스트

In [32]:
review1 = ' 운 좋게 시사회로 봤는데 진짜 재밌었음.. 올해 본 영화중 최고였음 ㅠ 문화의 날로 할인받아서 엄마랑 볼려고 또 예매함ㅎ'
review2 = '그냥 미쳤다. 뭐 하나 빠지는게 없다. 101마리 달마시안 팬들은 영화가 끝난것같아도 나가지말고 일단 앉아있으세요 ㅋㅋㅋㅋ?'

In [37]:
review3 = '세상에.. 정말 오랜만에 보는 영화라 기대했는데 기대 이하 정도가 아니라 이 영화를 보느라 할애한 제 귀중한 시간이 아까울 정도였습니다. 유치한 신파 요소는 그렇다 쳐도 영국 배경인데 그 흔한 유머 단 한마디도 없이...'

In [39]:
review4='세계에서 제일 매력 있는 빌런 영화'

In [5]:
type(review1)

str

In [12]:
# 이렇게하면 안됨..이유가 뭘까
review1 = review1.replace('[^ㄱ-ㅎㅏ-ㅣ가-힣 ]','')
review1

' 운 좋게 시사회로 봤는데 진짜 재밌었음 올해 본 영화중 최고였음 ㅠ 문화의 날로 할인받아서 엄마랑 볼려고 또 예매함ㅎ'

In [33]:
# 다른 방법으로 해결
import re
review1 = re.sub('[^ㄱ-ㅎㅏ-ㅣ가-힣 ]', '', review1)
review1

' 운 좋게 시사회로 봤는데 진짜 재밌었음 올해 본 영화중 최고였음 ㅠ 문화의 날로 할인받아서 엄마랑 볼려고 또 예매함ㅎ'

In [34]:
morphs = okt.morphs(review1)
review = ' '.join([word for word in morphs if not word in stopwords])
review

'운 좋게 시사회 로 봤는데 진짜 재밌었음 올해 본 영화 중 최고 였음 ㅠ 문화 날로 할인 받아서 엄마 랑 볼려고 또 예매 함 ㅎ'

In [35]:
review_cv = cvect.transform([review])  # 리스트로 만들어서 넣기
pred = lr.predict(review_cv)
print(pred)
print(pred[0])  # -- 결과: 1을 0으로 예측

[0]
0


In [36]:
# review2 에 대해 동일한 과정을 진행 -- 결과: 1을 0으로 예측

review2 = re.sub('[^ㄱ-ㅎㅏ-ㅣ가-힣 ]', '', review2)
morphs = okt.morphs(review2)
review = ' '.join([word for word in morphs if not word in stopwords])
review_cv = cvect.transform([review]) 
pred = lr.predict(review_cv)
print(pred)
print(pred[0])

[0]
0


In [38]:
# review3 에 대해 동일한 과정을 진행 -- 결과: 0을 0으로 예측

review3 = re.sub('[^ㄱ-ㅎㅏ-ㅣ가-힣 ]', '', review3)
morphs = okt.morphs(review3)
review = ' '.join([word for word in morphs if not word in stopwords])
review_cv = cvect.transform([review]) 
pred = lr.predict(review_cv)
print(pred)
print(pred[0])

[0]
0


In [40]:
# review4 에 대해 동일한 과정을 진행 -- 결과: 1을 1로 예측

review4 = re.sub('[^ㄱ-ㅎㅏ-ㅣ가-힣 ]', '', review4)
morphs = okt.morphs(review4)
review = ' '.join([word for word in morphs if not word in stopwords])
review_cv = cvect.transform([review]) 
pred = lr.predict(review_cv)
print(pred)
print(pred[0])

[1]
1


- 여러개의 리뷰가 리스트로 주어졌을 때

In [41]:
reviews = ['아름다운 음악과 아름다운 풍광~ 그렇지 못한 현실이 찡하네요~',
'메시지와 작위성의 불협화음!!!']

In [43]:
review_list = []
for review in reviews:
    review = re.sub('[^ㄱ-ㅎㅏ-ㅣ가-힣 ]', '', review)
    morphs = okt.morphs(review, stem=True)
    temp_X = ' '.join([word for word in morphs if not word in stopwords])
    review_list.append(temp_X)

In [44]:
review_list

['아름답다 음악 아름답다 풍 광 그렇다 못 현실 찡하다', '메시지 작위 성의 불협화음']

In [45]:
review_cv = cvect.transform(review_list)
pred = lr.predict(review_cv)
pred

array([1, 0], dtype=int64)

### 6. GridSearchCV로 최적 파라미터 찾기

In [46]:
from sklearn.pipeline import Pipeline
from sklearn.model_selection import GridSearchCV

In [47]:
pipeline = Pipeline([
    ('cvect', CountVectorizer()),
    ('lr', LogisticRegression())
])

params = { 'cvect__ngram_range':[(1,1),(1,2)],
            'cvect__max_df':[0.9, 0.99],
            'cvect__min_df':[1, 3],
            'lr__C':[1,5]
}

In [48]:
grid_pipe = GridSearchCV(
    pipeline, param_grid=params, cv=3, n_jobs=-1, scoring='accuracy'
)
%time grid_pipe.fit(X_train, y_train)
print(grid_pipe.best_params_)
print(grid_pipe.best_score_)

Wall time: 2min 9s
{'cvect__max_df': 0.9, 'cvect__min_df': 1, 'cvect__ngram_range': (1, 2), 'lr__C': 1}
0.8411836121571291


In [49]:
pred = grid_pipe.best_estimator_.predict(X_test)
accuracy_score(y_test, pred)

0.8466374119808143